In [1]:
# import nltk
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')

In [2]:
from reader import *
from string_matching_by_word import *
from string_matching_by_spaCy_NP import *
# from word_embeddings_by_spaCy_NP import *
from semantics_by_spaCy_NP import *
from hobbs import entry
from nltk import pos_tag

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
file_number = 'b19'

### Read Document

In [5]:
input_path = '/Users/ambuj/Documents/MS Stuff/nlp_cs_6340/final_project/nlp-project2019/dev/{}.input'.format(file_number)
key_path = '/Users/ambuj/Documents/MS Stuff/nlp_cs_6340/final_project/nlp-project2019/dev/{}.key'.format(file_number)

In [6]:
read = ReadInput(input_path)
list_of_sentences = read.getListOfSentences()
full_text = read.getText()

### Get Sentence Dict and Cluster Head Dict

In [7]:
sentence_dict = getSentenceDict(list_of_sentences)
cluster_head_dict = getClusterHeads(sentence_dict)

### Get Coref for exact match by word

In [8]:
coref_dict_match_word = getCorefDict_match_word(sentence_dict, cluster_head_dict,90)
# coref_dict_match_word

### Get Coref for threshold match by NP

In [9]:
coref_dict_match_NP = getCorefDict_match_NP(sentence_dict, cluster_head_dict,80)
# coref_dict_match_NP

### Get Coref for Word Embedding Similarity by NP

In [10]:
# coref_dict_all_sorted = getCorefDict_all_sorted(sentence_dict, cluster_head_dict,0.5)
# coref_dict_max_sentence = getCorefDict_max_of_each_sentence(sentence_dict, cluster_head_dict,0.5)

In [11]:
# coref_dict_all_sorted_top3 = get_TopN_Matches(coref_dict_all_sorted,3)

### Merging dictionaries

In [12]:
def mergeDicts(dict1, dict2):
    
    d1 = dict1.copy()
    d2 = dict2.copy()
    
    d1_keys = d1.keys()
    d2_keys = d2.keys()
    
    for key in d1_keys:
        d1[key] = list(map(lambda x: x[0:3], d1[key]))
    
    for key in d2_keys:
        d2[key] = list(map(lambda x: x[0:3], d2[key]))
    
    for key in d2_keys:
        if(key not in d1_keys):
            d1[key] = d2[key]
        else:
            for val in d2[key]:
                if(val not in d1[key]):
                    d1[key].append(val)
    
    return dict(sorted(d1.items()))

In [13]:
coref_final = mergeDicts(coref_dict_match_word, coref_dict_match_NP)

In [14]:
coref_final

{'X0': [[Early Aluminum Talks, 5, 'early contract talks'],
  [Early Aluminum Talks, 11, 'the early talks']],
 'X1': [[Steel Union, 5, 'The United Steelworkers union'],
  [Steel Union, 8, "the union's chief negotiator"],
  [Steel Union, 9, 'union members'],
  [Steel Union, 12, 'the Aluminum, Brick and Glass Workers union'],
  [Steel Union, 13, 'the two unions']],
 'X10': [[Pittsburgh-based Alcoa,
   13,
   'about 21,000 Alcoa and Reynolds employees']],
 'X11': [[Reynolds, Richmond, Va.,,
   13,
   'about 21,000 Alcoa and Reynolds employees']],
 'X12': [['union', 12, 'union'], ['union', 13, 'unions']],
 'X3': [[ST. LOUIS, 5, 'The United Steelworkers union'],
  [ST. LOUIS, 9, 'the aluminum industry'],
  [ST. LOUIS, 10, 'normal circumstances'],
  [ST. LOUIS, 12, 'the Steelworkers']],
 'X4': [['1986', 8, '1986']],
 'X6': [[our members, 9, 'union members']],
 'X7': [[sacrifices to help the companies over hard times,
   9,
   'the sacrifices']],
 'X8': [[George Becker, , 9, 'Mr. Becker']]}

### Passing the reference dict to Hobbs

In [15]:
coref_final_with_pro = entry(list_of_sentences, cluster_head_dict, coref_final)

In [16]:
coref_final_with_pro

{'X0': [[Early Aluminum Talks, 5, 'early contract talks'],
  [Early Aluminum Talks, 11, 'the early talks']],
 'X1': [[Steel Union, 5, 'The United Steelworkers union'],
  [Steel Union, 8, "the union's chief negotiator"],
  [Steel Union, 9, 'union members'],
  [Steel Union, 12, 'the Aluminum, Brick and Glass Workers union'],
  [Steel Union, 13, 'the two unions']],
 'X10': [[Pittsburgh-based Alcoa,
   13,
   'about 21,000 Alcoa and Reynolds employees']],
 'X11': [[Reynolds, Richmond, Va.,,
   13,
   'about 21,000 Alcoa and Reynolds employees']],
 'X12': [['union', 12, 'union'], ['union', 13, 'unions']],
 'X3': [[ST. LOUIS, 5, 'The United Steelworkers union'],
  [ST. LOUIS, 9, 'the aluminum industry'],
  [ST. LOUIS, 10, 'normal circumstances'],
  [ST. LOUIS, 12, 'the Steelworkers']],
 'X4': [['1986', 8, '1986']],
 'X6': [[our members, 9, 'union members']],
 'X7': [[sacrifices to help the companies over hard times,
   9,
   'the sacrifices']],
 'X8': [[George Becker, , 9, 'Mr. Becker']]}

### Adding those Cluster Heads which were not predicted

In [17]:
def addNOPredsClusterHeads(cluster_head_dict, coref_final_with_pro):
    for cluster_id, cluster_val in cluster_head_dict.items():
        if(cluster_id not in coref_final_with_pro.keys()):
            coref_final_with_pro[cluster_id] = [[cluster_val[1],int(cluster_val[0]),'', None]]
            
    return coref_final_with_pro

In [18]:
coref_final_with_pro = addNOPredsClusterHeads(cluster_head_dict, coref_final_with_pro)

In [19]:
coref_final_with_pro

{'X0': [[Early Aluminum Talks, 5, 'early contract talks'],
  [Early Aluminum Talks, 11, 'the early talks']],
 'X1': [[Steel Union, 5, 'The United Steelworkers union'],
  [Steel Union, 8, "the union's chief negotiator"],
  [Steel Union, 9, 'union members'],
  [Steel Union, 12, 'the Aluminum, Brick and Glass Workers union'],
  [Steel Union, 13, 'the two unions']],
 'X10': [[Pittsburgh-based Alcoa,
   13,
   'about 21,000 Alcoa and Reynolds employees']],
 'X11': [[Reynolds, Richmond, Va.,,
   13,
   'about 21,000 Alcoa and Reynolds employees']],
 'X12': [['union', 12, 'union'], ['union', 13, 'unions']],
 'X3': [[ST. LOUIS, 5, 'The United Steelworkers union'],
  [ST. LOUIS, 9, 'the aluminum industry'],
  [ST. LOUIS, 10, 'normal circumstances'],
  [ST. LOUIS, 12, 'the Steelworkers']],
 'X4': [['1986', 8, '1986']],
 'X6': [[our members, 9, 'union members']],
 'X7': [[sacrifices to help the companies over hard times,
   9,
   'the sacrifices']],
 'X8': [[George Becker, , 9, 'Mr. Becker']],
 '

### Making semantic prediction for unpredicted Cluster Heads

In [20]:
# import spacy
# nlp = spacy.load("en_core_web_lg")

In [21]:
# def getRemText(sent_id, sentence_dict):
#     rem_text = {}
#     for sid in sentence_dict.keys():
#         if(sid >= sent_id): 
#             rem_text[sid] = sentence_dict[sid]
#     sorted_rem_text = dict(sorted(rem_text.items(), key = lambda x:x[0]))
#     return sorted_rem_text

# def getCorefDict_meaning_NP(sentence_dict, coref_final_with_pro, threshold):
    
#     for cluster in coref_final_with_pro.keys():

#         if(coref_final_with_pro[cluster][0][2]==''):
    
#             current_cluster_sent_id = coref_final_with_pro[cluster][0][1]

#             print(coref_final_with_pro[cluster][0][0],'&'*8)
            
#             current_cluster_head = nlp(coref_final_with_pro[cluster][0][0].lower())
#             current_cluster_head_np = list(current_cluster_head.noun_chunks)
#             print(current_cluster_head_np)
            

#             if(len(current_cluster_head_np)==0):
#                 continue
            
# #             current_cluster_head_headNoun = current_cluster_head_np[0][-1]

#             remaining_text = getRemText(current_cluster_sent_id, sentence_dict)

#             for sid, sentence in remaining_text.items():

#                 clean = re.compile('<COREF .*?>.*?</COREF>') ## removing all coref tags from the current sentence
#                 clean_sentence = re.sub(clean, '', sentence)
#                 print('cl',clean_sentence)

#                 doc = nlp(clean_sentence.lower())
#                 print('doc',doc)

#                 for np in doc.noun_chunks:

#                     ## If the NP is a pronoun, ignore it. 
#                     ## We are handling pronouns by Hobb's algorithm
#                     if(len(np) == 1 and np[0].pos_ == 'PRON'):
#                         continue

#                     similarity = current_cluster_head_np[0].similarity(np)

#                     if(similarity*100 > threshold):
#                         print(np, current_cluster_head_np, similarity) 
                        
# #                         coref_final_with_pro[cluster][0][1] = sid
# #                         coref_final_with_pro[cluster][0][2] = np.text
# #                         coref_final_with_pro[cluster][0][3] = similarity*100   
    
#     return coref_final_with_pro

In [22]:
coref_final_with_pro = getCorefDict_meaning_NP(sentence_dict, coref_final_with_pro, 70)
coref_final_with_pro

{'X0': [[Early Aluminum Talks, 5, 'early contract talks'],
  [Early Aluminum Talks, 11, 'the early talks']],
 'X1': [[Steel Union, 5, 'The United Steelworkers union'],
  [Steel Union, 8, "the union's chief negotiator"],
  [Steel Union, 9, 'union members'],
  [Steel Union, 12, 'the Aluminum, Brick and Glass Workers union'],
  [Steel Union, 13, 'the two unions']],
 'X10': [[Pittsburgh-based Alcoa,
   13,
   'about 21,000 Alcoa and Reynolds employees']],
 'X11': [[Reynolds, Richmond, Va.,,
   13,
   'about 21,000 Alcoa and Reynolds employees']],
 'X12': [['union', 12, 'union'], ['union', 13, 'unions']],
 'X3': [[ST. LOUIS, 5, 'The United Steelworkers union'],
  [ST. LOUIS, 9, 'the aluminum industry'],
  [ST. LOUIS, 10, 'normal circumstances'],
  [ST. LOUIS, 12, 'the Steelworkers']],
 'X4': [['1986', 8, '1986']],
 'X6': [[our members, 9, 'union members']],
 'X7': [[sacrifices to help the companies over hard times,
   9,
   'the sacrifices']],
 'X8': [[George Becker, , 9, 'Mr. Becker']],
 '

### Taking only the head nouns

In [23]:
def takeHeadNouns(coref_final_with_pro):
    
    for cluster in coref_final_with_pro.keys():
        for i in range(0, len(coref_final_with_pro[cluster])):
            coref_final_with_pro[cluster][i][2] = coref_final_with_pro[cluster][i][2].split(' ')[-1]
                        
    return coref_final_with_pro

In [24]:
coref_final_with_pro = takeHeadNouns(coref_final_with_pro)

In [25]:
coref_final_with_pro

{'X0': [[Early Aluminum Talks, 5, 'talks'],
  [Early Aluminum Talks, 11, 'talks']],
 'X1': [[Steel Union, 5, 'union'],
  [Steel Union, 8, 'negotiator'],
  [Steel Union, 9, 'members'],
  [Steel Union, 12, 'union'],
  [Steel Union, 13, 'unions']],
 'X10': [[Pittsburgh-based Alcoa, 13, 'employees']],
 'X11': [[Reynolds, Richmond, Va.,, 13, 'employees']],
 'X12': [['union', 12, 'union'], ['union', 13, 'unions']],
 'X3': [[ST. LOUIS, 5, 'union'],
  [ST. LOUIS, 9, 'industry'],
  [ST. LOUIS, 10, 'circumstances'],
  [ST. LOUIS, 12, 'Steelworkers']],
 'X4': [['1986', 8, '1986']],
 'X6': [[our members, 9, 'members']],
 'X7': [[sacrifices to help the companies over hard times, 9, 'sacrifices']],
 'X8': [[George Becker, , 9, 'Becker']],
 'X2': [['2 Firms', 1, '', None]],
 'X5': [['The current pacts with Aluminum Co. of America and Reynolds Metals Co.',
   8,
   'contracts',
   81.36343955993652]],
 'X9': [['a strong recovery', 12, 'decision', 70.71035504341125]]}

### Print Output

In [26]:
def printOP(cluster_head_dict, coref_final_with_pro):
    
    for cluster_id, cluster_head_name in cluster_head_dict.items():

        print('<COREF ID="{}">{}</COREF>'.format(cluster_id, cluster_head_name[1]))

        coreferences = coref_final_with_pro[cluster_id]
        list_of_sent_ids = list(map(lambda x: x[1], coreferences))
        sorted_index_sent_ids = [i[0] for i in sorted(enumerate(list_of_sent_ids), key=lambda x:x[1])]
        coreferences = [coreferences[i] for i in sorted_index_sent_ids]

        for coref in coreferences:
            if(coref[2] == ''):
                continue
            print('{{{0}}}'.format(coref[1]) + ' ' + '{' + coref[2] + '}')
        print('\n', end = '')      

In [27]:
# %%capture cap --no-stderr
printOP(cluster_head_dict, coref_final_with_pro)

<COREF ID="X0">Early Aluminum Talks</COREF>
{5} {talks}
{11} {talks}

<COREF ID="X1">Steel Union</COREF>
{5} {union}
{8} {negotiator}
{9} {members}
{12} {union}
{13} {unions}

<COREF ID="X2">2 Firms</COREF>

<COREF ID="X3">ST. LOUIS</COREF>
{5} {union}
{9} {industry}
{10} {circumstances}
{12} {Steelworkers}

<COREF ID="X4">1986</COREF>
{8} {1986}

<COREF ID="X5">The current pacts with Aluminum Co. of America and Reynolds Metals Co.</COREF>
{8} {contracts}

<COREF ID="X6">our members</COREF>
{9} {members}

<COREF ID="X7">sacrifices to help the companies over hard times</COREF>
{9} {sacrifices}

<COREF ID="X8">George Becker, </COREF>
{9} {Becker}

<COREF ID="X9">a strong recovery</COREF>
{12} {decision}

<COREF ID="X10">Pittsburgh-based Alcoa</COREF>
{13} {employees}

<COREF ID="X11">Reynolds, Richmond, Va.,</COREF>
{13} {employees}

<COREF ID="X12">union</COREF>
{12} {union}
{13} {unions}



In [28]:
# with open('{}.response'.format(file_number),'w') as f:
#     f.write(cap.stdout)

### Answer Key

In [25]:
# reader_o = ReadInput(key_path)
# ans = reader_o.getListOfSentences()
# ans